In [1]:
import pandas as pd
import googleapiclient.discovery
import googleapiclient.errors

from pprint import pprint

In [2]:
api_service_name = "youtube"
api_version = "v3"
api_key = open("credentials/api_key.txt", "r").read()

# List of channels to search videos for

channel_list_name = ['GQ','Vogue','Vanity Fair']
channel_ids = ['UCsEukrAd64fqA7FjwkmZ_Dw','UCRXiA3h1no_PFkb1JCP0yMA','UCIsbLox_y9dCIMLd8tdC6qg']

# Request from YouTube using api_key
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=api_key)


In [3]:
def get_channel_stats(youtube, channel_ids):

    all_data = []
    request = youtube.channels().list(part="snippet,contentDetails,statistics",
                                      id=",".join(channel_ids),
                                      maxResults=25)
    response = request.execute()
    
    
    
    for item in response['items']:
        data = {'channelTitle' : item['snippet']['title'],
                'publishedAt'  : item['snippet']['publishedAt'],
                'playlistId'   : item['contentDetails']['relatedPlaylists']['uploads'],
                'subscribers'  : item['statistics']['subscriberCount'],
                'views'        : item['statistics']['viewCount'],
                'videos'       : item['statistics']['videoCount']}
        
        all_data.append(data)
        
        
        
    return(pd.DataFrame(all_data))
    

In [4]:
channel_data = get_channel_stats(youtube, channel_ids)

In [5]:
channel_data

,channelTitle,publishedAt,playlistId,subscribers,views,videos
0,Vogue,2008-06-29T22:48:58Z,UURXiA3h1no_PFkb1JCP0yMA,13100000,4080593719,3104
1,GQ,2008-01-16T16:12:23Z,UUsEukrAd64fqA7FjwkmZ_Dw,7400000,3058898636,2098
2,Vanity Fair,2006-08-09T22:09:20Z,UUIsbLox_y9dCIMLd8tdC6qg,4060000,2103887517,2658


In [6]:
def get_video_data(youtube,channel_data):
    
    all_data = []
    

    for playlistId in channel_data['playlistId']:
        request_playlistItems = youtube.playlistItems().list(part= "snippet,contentDetails",
                                                             playlistId=playlistId,
                                                             maxResults=50)
        response_playlistItems = request_playlistItems.execute()
        
        
        
        for item_playlist in response_playlistItems['items']: 
            request_videos = youtube.videos().list(part='snippet,contentDetails,statistics',
                                                   id=item_playlist['contentDetails']['videoId'],
                                                   maxResults=50)
            response_videos = request_videos.execute()
            
            
            
            for item_videos in response_videos['items']:
                data = {'channelId'   : item_videos['snippet']['channelId'],
                        'videoId'     : item_playlist['contentDetails']['videoId'],
                        'videoTitle'  : item_videos['snippet']['title'],
                        'publishedAt' : item_videos['snippet']['publishedAt'],
                        'duration'    : item_videos['contentDetails']['duration'],
                        'viewCount'   : item_videos['statistics']['viewCount'],
                        'likeCount'   : item_videos['statistics']['likeCount'],
                        'commentCount': item_videos['statistics']['commentCount'],
                        'favCount'    : item_videos['statistics']['favoriteCount']}
                
                all_data.append(data)
                
                
    return(pd.DataFrame(all_data))

In [7]:
video_data = get_video_data(youtube,channel_data)

In [8]:
video_data

,channelId,videoId,videoTitle,publishedAt,duration,viewCount,likeCount,commentCount,favCount
0,UCRXiA3h1no_PFkb1JCP0yMA,GFrw11vay9k,Priyanka Chopra's Daughter Eats Just Like Mom,2023-05-24T20:03:02Z,PT20S,59840,3759,39,0
1,UCRXiA3h1no_PFkb1JCP0yMA,LAz1ATucm3Q,"Margot Robbie Breaks Down 15 Looks, From The W...",2023-05-24T12:01:04Z,PT10M,304174,13008,260,0
2,UCRXiA3h1no_PFkb1JCP0yMA,onTAocTAfLk,Jennifer Lawrence ❤️ NY,2023-05-23T17:51:11Z,PT36S,46628,2225,19,0
3,UCRXiA3h1no_PFkb1JCP0yMA,EJya33R0wZI,Alia Bhatt Arrives at the Gucci Cruise 2024 Fa...,2023-05-22T19:01:01Z,PT26S,84139,5081,26,0
4,UCRXiA3h1no_PFkb1JCP0yMA,u_kFNzvdCqA,Alia Bhatt Gets Ready For Her First Fashion Show,2023-05-22T16:04:23Z,PT58S,93082,5571,42,0
...,...,...,...,...,...,...,...,...,...
145,UCIsbLox_y9dCIMLd8tdC6qg,88MCNN0abIU,"Willem Dafoe Rewatches Spider-Man, The Lightho...",2023-03-20T16:00:02Z,PT12M46S,300497,12107,334,0
146,UCIsbLox_y9dCIMLd8tdC6qg,mDTWUKJy_XI,Jenna Ortega & 'Scream 6' Cast Test How Well T...,2023-03-17T16:00:01Z,PT10M46S,886697,25237,964,0
147,UCIsbLox_y9dCIMLd8tdC6qg,i0pPWFgJNNk,Halsey & Amelia Dimoldenberg on Dating Each Ot...,2023-03-15T19:00:04Z,PT39S,41780,1226,15,0
148,UCIsbLox_y9dCIMLd8tdC6qg,Yu2lOJjzH6Y,Pedro Pascal is someone who dances 🕺🏻,2023-03-15T17:00:03Z,PT33S,47471,2425,27,0
